In [1]:
#do this later from admin: !python -m pip install --upgrade pip


# TOC

0. [Imports](#first-bullet)
1. [Functions](#1.0-bullet)
2. [Main Pull](#2.0-bullet)
3. [Cleaning](#3.0-bullet)  
 3.1 [Pt 1 - Basic Cleaning](#3.1-bullet)  
 3.2 [Pt 2 - Advanced Cleaning and feature engineering](#3.2-bullet)
4. [Reference and Planning](#4.0-bullet)

# Do not Run All cells - lots of intermittent cleaning included for reference

Jump to Cleanup

In [1]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#Youtube Data API Packages: 
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
from googleapiclient.discovery import build
import argparse
"""
from googleapiclient.errors import HttpError"""

#Modeling
#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

#NLP & Text Management
'''from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer'''
import re

#Time analysis
import time
import datetime
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('seaborn-whitegrid')
sns.set_style('whitegrid')

In [10]:
my_api_file = open("./mykey.txt", 'r')
my_api_str = my_api_file.read()
my_api_file.close()

In [11]:
DEVELOPER_KEY = my_api_str
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [12]:
def youtube_search_cat(q, max_results=50, sort_method='viewCount', token=None):
    # Call the search.list method to retrieve results matching the specified query term.
    #videoCategoryId is cat 10 - music
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)
    
    search_response = youtube.search().list(
        q=q,
        part='id,snippet',
        pageToken=token,
        maxResults=max_results,
        order=sort_method,
        videoCategoryId='10',
        type='video'
    ).execute()
    
    return search_response

In [13]:
#takes output json list from search list and returns metadata on individual videos
def youtube_search_video_cat(q='spinners', max_results=50, sort_method='viewCount', token=None):
    
    order = "viewCount"
    q=q
    max_results = max_results
    sort_method = sort_method
    token = token
    location = None
    location_radius = None
    
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,         
    developerKey=DEVELOPER_KEY)
    
    #Return list of matching records up to max_search
    search_result = youtube_search_cat(q, max_results, sort_method, token)
    next_tok = search_result['nextPageToken']
    
    videos_list = []
    for search_result in search_result.get("items", []):
        
        if search_result["id"]["kind"] == 'youtube#video':
            temp_dict_ = {}
            #Available from initial search
            temp_dict_['title'] = search_result['snippet']['title']  
            temp_dict_['vidId'] = search_result['id']['videoId']  
            
            #Secondary call to find statistics results for individual video
            response = youtube.videos().list(
                part='statistics, snippet, contentDetails', 
                id=search_result['id']['videoId']
                    ).execute()
            #Relevant dictionaries
            response_statistics = response['items'][0]['statistics']
            response_snippet = response['items'][0]['snippet']
            response_content= response['items'][0]['contentDetails']
            
            
            snippet_list = ['publishedAt','channelId', 'description', 
                            'channelTitle', 'tags', 'categoryId', 
                            'liveBroadcastContent', 'defaultLanguage', ]
            for val in snippet_list:
                try:
                    temp_dict_[val] = response_snippet[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'    
            
            stats_list = ['favoriteCount', 'viewCount', 'likeCount', 
                          'dislikeCount', 'commentCount']
            for val in stats_list:
                try:
                    temp_dict_[val] = response_statistics[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
            
            for val in response_content.keys():
                try:
                    temp_dict_[val] = response_content[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
                    
            #add back to main list
            videos_list.append(temp_dict_)
            
    return videos_list, next_tok

In [14]:
#Automation
def youtube_search_video_looper_cat(q,max_results=50, 
                                sort_method='viewCount', 
                                token=None, 
                                dl_path='./data/test/', dl_title='iteration'):
    #find relevant stats, create dataframe of results, create running csvs
    q=q,
    max_results=max_results
    sort_method = sort_method
    token = token
    
    #create unique text file name
    now_month = datetime.datetime.now().month
    now_day = datetime.datetime.now().day
    now_hour = datetime.datetime.now().hour
    now_minute = datetime.datetime.now().minute
    output_title = '{}{}_{}.{}_{}{}.csv'.format(dl_path, dl_title,  
                                                  now_month, now_day, now_hour, now_minute)
    
    
    #Cool Start to get next token
    videos_list, next_tok = youtube_search_video_cat(q=q,
                                                 max_results=max_results, 
                                                 sort_method=sort_method, 
                                                 token=token)
    df_videos = pd.DataFrame(videos_list)
    df_videos['request_token'] = next_tok #for assessment of run.

    #Set up exit criteria - 2 counts in a row means its done
    previous_results = 0
    current_results = df_videos.shape[0]
    
    count=1
    while previous_results != current_results:
        try:

            videos_list, next_tok = youtube_search_video_cat(q,
                                                         max_results=max_results, 
                                                         sort_method=sort_method, 
                                                         token=next_tok)
            print('    found page ', count)
            df_small_vids = pd.DataFrame(videos_list)
            df_small_vids['request_token'] = next_tok
            df_videos = pd.concat([df_videos, df_small_vids], sort=False)


            #save as we go
            df_videos.to_csv(output_title, index=False)

            _temp = current_results
            current_results = df_videos.shape[0]
            previous_results = _temp
            count += 1
            time.sleep(2)
        except:
            break
    total_pulls = count
    return df_videos, next_tok, total_pulls
        

# <a class="anchor" id="2.0-bullet"> 2. Data Pull</a>

In [15]:
#This is the major looping item used to collect data for alphabet_pull_init and cat10_relevance
letters = 'abcdefghijklmnopqrxtuvwxyz'

df_big_videos = pd.DataFrame()

count=0
for letter in letters:
    df_videos, next_tok, total_pulls = youtube_search_video_looper_cat(
        q=letter, sort_method='relevance', dl_path='./data/cat10_relevance/', dl_title='search'+letter+'2')
    df_videos['letter_search'] = letter
    print('   foundall: ', letter)
    df_big_videos = pd.concat([df_big_videos, df_videos], sort=False)
    df_big_videos.to_csv('./data/cat10_relevance/bigletters2.csv')
    

    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
   foundall:  a
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
   foundall:  b
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
   foundall:  c
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
   foundall:  d
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
   foundall:  e
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
    found page  10
 

In [38]:
df_big_videos.shape

(10259, 25)

# <a class="anchor" id="3.0-bullet"> 3. Cleanup</a>

## <a class="anchor" id="3.1-bullet">3.1 Nulls and Stuff</a>

Note: This section was completed immediately after the main data pull was completed, where df_big_videos was still in memory.  
Progress was saved and continued in 3.2. 

In [2]:
df_big_videos = pd.read_csv('./data/cat10_relevance/bigletters2.csv')

In [3]:
df_new = df_big_videos

In [4]:
#Remove duplicate indices from each individual request (came in batches of about 320)
df_new = df_new.reset_index()
df_new = df_new.drop('index', axis=1)

In [5]:
df_new.columns

Index(['Unnamed: 0', 'caption', 'categoryId', 'channelId', 'channelTitle',
       'commentCount', 'defaultLanguage', 'definition', 'description',
       'dimension', 'dislikeCount', 'duration', 'favoriteCount',
       'licensedContent', 'likeCount', 'liveBroadcastContent', 'projection',
       'publishedAt', 'regionRestriction', 'tags', 'title', 'vidId',
       'viewCount', 'request_token', 'letter_search', 'contentRating'],
      dtype='object')

In [6]:
#Drop duplicates based on vidID - tags came in as a list, which the function can not otherwise handle.
df_new = df_new.drop_duplicates(subset='vidId')

In [7]:
#Convert nulls to 0, reform column as integers
df_new['commentCount'] = df_new['commentCount'].map(lambda x: 0 if x == 'xxNoneFoundxx' else x )
df_new['commentCount'] =df_new['commentCount'].astype('str').astype('int')
df_new['commentCount'].sum()

171454286

In [8]:
#General the above process
def clean_df_new_col(col):
    df_new[col] = df_new[col].map(lambda x: 0 if x == 'xxNoneFoundxx' else x )
    df_new[col] =df_new[col].astype('str').astype('int64')

clean_df_new_col('favoriteCount')
clean_df_new_col('likeCount')
clean_df_new_col('viewCount')
clean_df_new_col('dislikeCount')

In [9]:
#Convert tags lists to comma separated strings
df_new['tags'] = df_new.tags.apply(lambda x: ', '.join(x) if x != 'xxNoneFoundxx' else '')

In [154]:
#Save point 1
#df_new.to_csv('./data/alphabet_pull_init_01.10.18.csv', index=False)

## <a class="anchor" id="3.2-bullet">3.2 Interpretation and Advanced Cleaning</a>


### Duration Converstion

In [10]:
#df_data = pd.read_csv('./data/alphabet_pull_init_01.10.18.csv')
df_data = df_new
df_data.shape

(8670, 26)

In [11]:
df_temp = pd.DataFrame(df_data.loc[:, 'duration'])
#Regex testing
#Hours
test_string1 = 'PT3H5M1S'
print('    3=', int((re.findall('PT(.+)[01234567890]?H', test_string1))[0]))

#Minutes
test_string2 = 'PT32H21M1S'
print('    21=', int(re.findall('[A-Z]([01234567890]*?M)', test_string2)[0].replace('M', '')))

#Seconds
test_string2 = 'PT23H12S'
print('    12=', int(re.findall('[A-Z]([01234567890]*?S)', test_string2)[0].replace('S', '')))

    3= 3
    21= 21
    12= 12


In [12]:
#Find digits in string and create column to hold
df_temp['hours'] = df_temp['duration'].apply(lambda x: 
                                             0 if 'H' not in x else int((re.findall('PT(.+)[01234567890]?H', 
                                                                                    x))[0]))
df_temp['minutes'] = df_temp['duration'].apply(lambda x: 
                                             0 if 'M' not in x else int(re.findall('[A-Z]([01234567890]*?M)',
                                                                                   x)[0].replace('M', '')))
df_temp['seconds'] = df_temp['duration'].apply(lambda x: 
                                             0 if 'S' not in x else int(re.findall('[A-Z]([01234567890]*?S)',
                                                                                   x)[0].replace('S', '')))
df_temp['seconds_tot'] = df_temp['seconds'] + df_temp['minutes'] * 60 + df_temp['hours'] * 60 * 60

df_temp.head()

,duration,hours,minutes,seconds,seconds_tot
0,PT5M4S,0,5,4,304
1,PT6M3S,0,6,3,363
2,PT4M39S,0,4,39,279
3,PT3M54S,0,3,54,234
4,PT3M29S,0,3,29,209


In [13]:
df_data['duration'] = df_temp['seconds_tot'] #seems agressive to look too deeply into this.

### Publish Date

In [14]:
df_data.loc[:, ['publishedAt']] = pd.to_datetime(df_data.publishedAt)

In [15]:
df_data['year'] = df_data.publishedAt.dt.year - 2000
df_data['month'] = df_data.publishedAt.dt.month
df_data['month_day'] = df_data.publishedAt.dt.day
df_data['year_day'] = df_data.publishedAt.dt.dayofyear
df_data['week_day'] = df_data.publishedAt.dt.dayofweek

### Get Dummies 
 * definition, (0 for sd, 1 for hd)
 * licensedContent (map)
 * Caption (map)

In [16]:
df_data.loc[:, ['definition']] = df_data.definition.map({'sd':0, 'hd':1})

In [17]:
df_data.loc[:, ['licensedContent']] = df_data.licensedContent.map({False:0, True:1})

In [18]:
df_data.loc[:, ['caption']] = df_data.caption.map({False:0, True:1})

### Drop, convert

In [19]:
cols_to_drop = ['categoryId', 'channelId', 'dimension', 'liveBroadcastContent', 'projection']
#Changing this since I am destroying detail for the last columns, and for the drops
df_clean = df_data.loc[:, [col for col in df_data.columns if col not in cols_to_drop]]
df_clean['regionRestriction'] = df_clean['regionRestriction'].fillna(0).map(lambda x: 1 if x!= 0 else 0)
df_clean['contentRating'] = df_clean['contentRating'].fillna(0).map(lambda x: 1 if x!= 0 else 0)

In [27]:
df_clean = df_clean.drop('Unnamed: 0', axis=1)

In [20]:
meta_cols = ['vidId','request_token', 'letter_search', ]
cols_to_hold = ['defaultLanguage', 'regionRestriction','contentRating' ] #These are pretty sparse
cols_for_analysis = [  'duration', 'licensedContent', 'definition',   #other 
                     'caption', 'channelTitle','tags', 'title', 'description', #text 
                     'publishedAt', 'year', 'month','month_day', 'year_day', 'week_day',  #dates
                     'viewCount','likeCount','dislikeCount', 'favoriteCount','commentCount', #Results
                    ]


In [28]:
len(cols_to_hold) + len(cols_for_analysis) + len(meta_cols) == len(df_clean.columns)

True

In [29]:
[col for col in df_clean.columns if col not in meta_cols and col not in cols_to_hold and col not in cols_for_analysis]

[]

In [30]:
#df_data.to_csv('./data/alphabet_pull_round2_02.10.18.csv', index=False)
df_clean.to_csv('./data/clean_data_nocomments_noviews_10.10.18.csv', index=False)